<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [2]'.</span>

<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [2]'.</span>

<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [2]'.</span>

<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [2]'.</span>

# Hello Treebeard!

Thanks for trying out Treebeard's cloud deploy and scheduling service.  


1. [Requirements](#Requirements)
2. [Outputs](#Outputs)
3. [Examples](#Examples)
    - [Call an API](#Call-an-API)
    - [Check a hosted file](#Check-a-hosted-file)

## Requirements

In [1]:
import treebeard
# This notebook does not depend on the treebeard library, but it should be present in your requirements file.
# The underlying build service sends this notebook to a cloud server which uses the papermill library to run
# the notebook. Having treebeard as a dependency ensure this functions correctly.

There must be a dependencies file alongside this notebook, either requirements.txt, a pipenv Pipfile, or a Conda environment.yml file.

In [2]:
# Setup
import os
import matplotlib.pyplot as plt 
import requests
import json
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

ModuleNotFoundError: No module named 'seaborn'

## Output

**Output** from your notebooks should be saved in an `output` directory.  
If so, they will be saved on the cloud build server, versioned, and made available via URL endpoints.  
They can be listed on the Treebeard admin page that you can find after running a build.

In [ ]:
# create an outputs directory if it does not exist
if 'output' not in os.listdir(): os.mkdir('output')

# Examples

Here are a few simple examples of data that might be interesting to check over time.

## Call an API

What's the current price of BitCoin?

In [ ]:
r = requests.get('https://api.coindesk.com/v1/bpi/currentprice.json').json()
print(f"At {r['time']['updateduk']}, the price of Bitcoin is £{r['bpi']['GBP']['rate']}")
# This data was produced from the CoinDesk Bitcoin Price Index (USD). Non-USD currency data converted using hourly conversion rate from openexchangerates.org

See the last 30 days of the price of Bitcoin, thanks to CoinDesk's API

In [ ]:
r = requests.get('https://api.coindesk.com/v1/bpi/historical/close.json').json()

fig, ax = plt.subplots()
ax.plot(pd.DataFrame.from_dict(r)['bpi'][:-2], color='red', linestyle='--')
ax.set_ylabel('Bitcoin Price (£)')
fig.suptitle('Last 30 Days of Bitcoin Price', size=16)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
ax.xaxis.set_major_locator(plt.MaxNLocator(4))

im = plt.imread('https://emojipedia-us.s3.dualstack.us-west-1.amazonaws.com/thumbs/320/apple/237/money-with-wings_1f4b8.png')
newax = fig.add_axes([0.8, 0.8, 0.2, 0.2], anchor='NE')
newax.imshow(im)
newax.axis('off')
plt.savefig('output/bitcoin_price.png', dpi=150, bbox_inches='tight')

## Check a hosted file

The [Mauna Loa Observatory](https://www.esrl.noaa.gov/gmd/obop/mlo/) is a USA NOAA Earth System Research Laboratory.  
It publishes measurements of **atmospheric CO2 concentration** regularly.  
The Observatory publishes its own charts [here](https://www.esrl.noaa.gov/gmd/ccgg/trends/mlo.html), but we can recreate them.

In [ ]:
# Weekly CO2 data is published here, there are also other datafeeds available
url = "ftp://aftp.cmdl.noaa.gov/products/trends/co2/co2_weekly_mlo.txt"

In [ ]:
# Pandas can read remote URL file contents
# Skip the first 49 rows as they are explanatory text
# Reading them is helpful though as they define the column names and missing number fills
df = pd.read_table(url, skiprows=49, header=None)

In [ ]:
df[0].head()

In [ ]:
# Use a regex to turn the variable whitespace between columns into commas
df[0] = df[0].str.strip().str.replace("(\s{1,})", ",")

In [ ]:
# Split the column by commas and expand into columns
df = df[0].str.split(',', expand=True)

In [ ]:
# Set the column headers
columns = ["yr", "mon", "day", "decimal", "ppm", "days", "1_yr_ago", "10_yr_ago", "since_1800"]
df.columns=columns

In [ ]:
# Replace -999.99 with NAs
df = df.replace('-999.99', np.nan)

In [ ]:
# Create datetime column
df['date'] = pd.to_datetime(dict(year=df['yr'], month=df['mon'], day=df['day']))

In [ ]:
# Set types
df['ppm'] = df['ppm'].apply(pd.to_numeric)

In [ ]:
df.tail()

In [ ]:
sns.set()
fig, ax = plt.subplots(figsize=(8,8))
sns.lineplot(x='date', y='ppm', data=df)
plt.title("Weekly Atmospheric CO2 concentration at Mauna Loa Observatory")
plt.savefig('output/co2_concentration.png', dpi=150, bbox_inches='tight')

## Call a python script

Notebooks often interact with custom scripts.  
The script `my_script.py` is called into this notebook and a function executed.

In [ ]:
# This depends on which directory you are running jupyter from

# from examples.hello_treebeard.my_script import greeter # project root
from my_script import greeter # or hello_treebeard

greeter('World')

## Running Treebeard

Ok, let's run this in the cloud!  
You'll need to run `treebeard configure` in the command line to register your account.  
For a new project you'll need to run `treebeard setup` in the project directory to create a `treebeard.yaml` file. However this has already been done. 

In the command line run `treebeard run` to compress the project folder, create a docker container and request a cloud build.  

You will receive a link to the admin page where you can see results after a few minutes.  
It is simple to add a schedule flag to this command: `treebeard run --daily`

The saved image outputs can be seen in the admin page, see [the documentation](https://treebeard.readthedocs.io/en/latest/outputs.html) for the URL schema to access them.